<a href="https://colab.research.google.com/github/moustafa-7/ChatBot-Project/blob/master/ECG_Heartbeat_Classification_1D_Convolutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-datasets/29414/37484/heartbeat.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1567102733&Signature=etKL7luuxEKCg7LvbNucUpRZlwHsDFp%2Fm4CD3O7brD9VoBR0tFM9tlIATVS00LNujJbMM0sy7KYrQEVfP%2BCOJBqpUvyJrpW%2BnlZhajWgW37W0zOBICJ8K1oRiUxwW0jeRThP8JqkqLpbDH1HxRWpy%2FmOEvOr1sc%2Fkod3W28gnc9AdzkaH9MV27tZVmgtWaIGtgr1ukvs5b5WFMXfAujR49Fhndy9VoY%2BUZSZbG8gaebDfqAafIU6E6DOUeHGV2mlSdGAQOjx%2FBd3WG36m924wovmORkaosXm59uCd9fg8F9Qz8YdXJKS5MfYTXbjYv2g9tXM1uBSt7xznhMMvovZ2Q%3D%3D" -O "heartbeat.zip" -c

--2019-08-26 18:19:15--  https://storage.googleapis.com/kaggle-datasets/29414/37484/heartbeat.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1567102733&Signature=etKL7luuxEKCg7LvbNucUpRZlwHsDFp%2Fm4CD3O7brD9VoBR0tFM9tlIATVS00LNujJbMM0sy7KYrQEVfP%2BCOJBqpUvyJrpW%2BnlZhajWgW37W0zOBICJ8K1oRiUxwW0jeRThP8JqkqLpbDH1HxRWpy%2FmOEvOr1sc%2Fkod3W28gnc9AdzkaH9MV27tZVmgtWaIGtgr1ukvs5b5WFMXfAujR49Fhndy9VoY%2BUZSZbG8gaebDfqAafIU6E6DOUeHGV2mlSdGAQOjx%2FBd3WG36m924wovmORkaosXm59uCd9fg8F9Qz8YdXJKS5MfYTXbjYv2g9tXM1uBSt7xznhMMvovZ2Q%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 2404:6800:4008:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103633608 (99M) [application/zip]
Saving to: ‘heartbeat.zip’

heartbeat.zip       100%[===================>]  98.83M  37.9MB/s    in 2.6s    

2019-08-26 18:19:18 (37.9 MB/s) - ‘heartbeat.zip’ save

In [2]:
!unzip heartbeat.zip -d data

Archive:  heartbeat.zip
  inflating: data/ptbdb_abnormal.csv  
  inflating: data/mitbih_train.csv   
  inflating: data/ptbdb_normal.csv   
  inflating: data/mitbih_test.csv    


In [0]:
import pandas as pd
import numpy as np

import keras
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler,ReduceLROnPlateau

from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate
  
import sklearn
from sklearn.metrics import f1_score, accuracy_score


In [0]:
df_train = pd.read_csv('data/mitbih_train.csv', header = None)

#To shuffle the training data.
df_train = df_train.sample(frac = 1)

# reading 
df_test = pd.read_csv('data/mitbih_test.csv', header = None)

In [0]:
# now transforming the data to an np array
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]
Y = np.array(df_train[187].values).astype(np.int8)

X_test = np.array(df_test[list(range(187))].values)[... , np.newaxis]
Y_test = np.array(df_test[187].values).astype(np.int8)


In [17]:
X[0].shape

(187, 1)

In [0]:
# now defining the model

def get_model():
  n_classes = 5
  input_data = Input(shape = [187,1],)
  
  inp = Convolution1D(16, kernel_size = 5, activation = activations.relu, padding= 'valid')(input_data)
  inp = Convolution1D(16, kernel_size = 5, activation = activations.relu, padding = 'valid')(inp)
  
  inp = MaxPool1D(pool_size = 2, padding = 'valid')(inp)
  
  inp = Dropout(rate = 0.1)(inp)
  
  inp = Convolution1D(32, kernel_size = 3, activation = activations.relu, padding = 'valid')(inp)
  inp = Convolution1D(32, kernel_size = 3, activation = activations.relu, padding = 'valid')(inp)
  
  inp = MaxPool1D(pool_size = 2)(inp)
  
  inp = Dropout(rate = 0.1)(inp)
  
  inp = Convolution1D(32, kernel_size = 3, activation = activations.relu, padding = 'valid')(inp)
  inp = Convolution1D(32, kernel_size = 3, activation = activations.relu, padding = 'valid')(inp)
  
  inp = MaxPool1D(pool_size = 2)(inp)
  
  inp = Dropout(rate = 0.1)(inp)
  
  inp = Convolution1D(256, kernel_size = 3, activation = activations.relu, padding = 'valid')(inp)
  inp = Convolution1D(256, kernel_size = 3, activation = activations.relu, padding = 'valid')(inp)
  
  inp = GlobalMaxPooling1D()(inp)
  
  inp = Dropout(rate = 0.1)(inp)
  
  dense_1 = Dense(64, activation = activations.relu, name = 'dense_1')(inp)
  dense_2 = Dense(64, activation = activations.relu, name = 'dense_2')(dense_1)
  
  dense_2 = Dense(n_classes, activation = activations.softmax, name = 'output')(dense_2)
  
  
  model = models.Model(inputs = input_data, outputs = dense_2)
  
  optimizer = optimizers.Adam(0.001) # learning_rate = 0.001
  
  model.compile(optimizer = optimizer, loss = losses.sparse_categorical_crossentropy, metrics = ['acc'])
  
  model.summary()
  
  return model
  

In [30]:
model = get_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 187, 1)            0         
_________________________________________________________________
conv1d_34 (Conv1D)           (None, 183, 16)           96        
_________________________________________________________________
conv1d_35 (Conv1D)           (None, 179, 16)           1296      
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 89, 16)            0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 89, 16)            0         
_________________________________________________________________
conv1d_36 (Conv1D)           (None, 87, 32)            1568      
_________________________________________________________________
conv1d_37 (Conv1D)           (None, 85, 32)            3104      
__________

In [0]:
file_path = "cnn_1d_mitbih.h5"

In [0]:
checkpoint = ModelCheckpoint(file_path, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')

In [0]:
early_stopping = EarlyStopping(monitor = 'val_acc', verbose = 1, mode = 'max', patience = 5)

In [0]:
red_on_plateau = ReduceLROnPlateau(monitor = 'val_acc', factor = 0.2, patience = 3, mode = 'max', min_lr = 0.001, verbose = 1)

In [0]:
callbacks_list = [checkpoint, early_stopping, red_on_plateau]

In [37]:
model.fit(X, Y, epochs = 1000, callbacks = callbacks_list, validation_split = 0.1)
model.load_weights(file_path)

Train on 78798 samples, validate on 8756 samples
Epoch 1/1000
78798/78798 [==============================] - 33s 414us/step - loss: 0.2819 - acc: 0.9214 - val_loss: 0.2024 - val_acc: 0.9468

Epoch 00001: val_acc improved from -inf to 0.94678, saving model to cnn_1d_mitbih.h5
Epoch 2/1000
78798/78798 [==============================] - 32s 410us/step - loss: 0.1746 - acc: 0.9518 - val_loss: 0.1474 - val_acc: 0.9583

Epoch 00002: val_acc improved from 0.94678 to 0.95831, saving model to cnn_1d_mitbih.h5
Epoch 3/1000
78798/78798 [==============================] - 32s 407us/step - loss: 0.1443 - acc: 0.9605 - val_loss: 0.1214 - val_acc: 0.9671

Epoch 00003: val_acc improved from 0.95831 to 0.96711, saving model to cnn_1d_mitbih.h5
Epoch 4/1000
78798/78798 [==============================] - 32s 406us/step - loss: 0.1284 - acc: 0.9651 - val_loss: 0.1144 - val_acc: 0.9688

Epoch 00004: val_acc improved from 0.96711 to 0.96882, saving model to cnn_1d_mitbih.h5
Epoch 5/1000
78798/78798 [========

In [39]:
pred_test = model.predict(X_test)
pred_test = np.argmax(pred_test, axis = -1)

f1 = f1_score(Y_test, pred_test, average = 'macro')

print("Test F1 score: {}".format(f1))

acc = accuracy_score(Y_test, pred_test)
print("Test accuracy: {}".format(acc))

Test F1 score: 0.902715327147407
Test accuracy: 0.982733418600402
